Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
Постройте модель со значением метрики качества F1 не меньше 0.75.

In [2]:
import ssl 
ssl._create_default_https_context = ssl._create_unverified_context

In [59]:
import torch
from torch import nn
import transformers
import pandas as pd
import numpy as np
import string
import nltk
import re
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem.snowball import EnglishStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from gensim.models import Word2Vec
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, make_scorer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_predict
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nikitazaytsev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nikitazaytsev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [60]:
data = pd.read_csv('./toxic_comments.csv')
data_lr = pd.read_csv('./toxic_comments.csv')

In [46]:
data.head(25)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [47]:
data.shape

(159571, 2)

In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [49]:
data.describe()

,toxic
count,159571.000000
mean,0.101679
std,0.302226
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [50]:
data.isnull().any()

text     False
toxic    False
dtype: bool

In [51]:
data.shape

(159571, 2)

In [52]:
data['text'] = data['text'].str.lower()

In [53]:
data['text'] = data['text'].apply(lambda x: re.sub(r'[^a-z@# ]', '', x))

In [54]:
# data

In [35]:
data['text'] = data['text'].apply(lambda word: word_tokenize(word))

In [36]:
stop_words = set(nltk_stopwords.words('english'))

In [37]:
data['text'] = data['text'].apply(lambda token_list: [word for word in token_list if word not in stop_words])

In [38]:
data.head(25)

,text,toxic
0,"[explanationwhy, edits, made, username, hardco...",0
1,"[daww, matches, background, colour, im, seemin...",0
2,"[hey, man, im, really, trying, edit, war, guy,...",0
3,"[morei, cant, make, real, suggestions, improve...",0
4,"[sir, hero, chance, remember, page, thats]",0
5,"[congratulations, well, use, tools, well, talk]",0
6,"[cocksucker, piss, around, work]",1
7,"[vandalism, matt, shirvington, article, revert...",0
8,"[sorry, word, nonsense, offensive, anyway, im,...",0
9,"[alignment, subject, contrary, dulithgow]",0


In [39]:
data['text'] = data['text'].apply(lambda tokens: ' '.join(tokens))

In [40]:
X = TfidfVectorizer().fit_transform(data['text'])
Y = data['toxic']

In [41]:
X.shape

(159571, 273049)

In [42]:
Y.shape

(159571,)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

Модель опорного вектора

In [55]:
model = LinearSVC()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred)
print(f"Метрика качества F1 на тестовых данных: {f1}")

f1_scorer = make_scorer(f1_score)
f1_scores = cross_val_score(model, X_train, y_train, cv=5, scoring=f1_scorer)

print(f"Метрика качества F1: {f1_scores.mean()}")

Метрика качества F1 на тестовых данных: 0.7602256699576869
Метрика качества F1: 0.7673466117439721


Модель линейной регрессии

In [71]:
data_lr["text"] = data_lr["text"].apply(lambda x: x.lower())

x_vector = TfidfVectorizer().fit_transform(data_lr['text'])

y = data_lr['toxic']

x_train, x_test, y_train, y_test = train_test_split(x_vector, y, test_size=0.2, random_state=42)

In [72]:
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred)
print(f"Метрика качества F1 на тестовых данных: ", f1)

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [69]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['toxic'], test_size=0.2, random_state=42)

stop_words = set(nltk_stopwords.words('english'))
def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

X_train = X_train.apply(preprocess)
X_test = X_test.apply(preprocess)

In [70]:
sentences = [sentence.split() for sentence in X_train]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)

def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train = np.array([vectorize(sentence) for sentence in X_train])
X_test = np.array([vectorize(sentence) for sentence in X_test])

clf = LogisticRegression()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

f1 = f1_score(y_test, y_pred)
print(f"Метрика качества F1 на тестовых данных: {f1}")

f1_scorer = make_scorer(f1_score)
f1_scores = cross_val_score(model, X_train, y_train, cv=5, scoring=f1_scorer)

print(f"Метрика качества F1: {f1_scores.mean()}")

Метрика качества F1 на тестовых данных: 0.6097560975609756


/Users/nikitazaytsev/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/nikitazaytsev/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_scorer.py", line 139, in __call__
    score = scorer._score(
  File "/Users/nikitazaytsev/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/Users/nikitazaytsev/Library/Python/3.9/lib/python/site-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "/Users/nikitazaytsev/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py", line 1279, in f1_score
    return fbeta_score(
  File "/Users/nikitazaytsev/Library/Python/3.9/lib/python/site-pac

Метрика качества F1: nan


/Users/nikitazaytsev/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/nikitazaytsev/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_scorer.py", line 139, in __call__
    score = scorer._score(
  File "/Users/nikitazaytsev/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_scorer.py", line 376, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/Users/nikitazaytsev/Library/Python/3.9/lib/python/site-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "/Users/nikitazaytsev/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py", line 1279, in f1_score
    return fbeta_score(
  File "/Users/nikitazaytsev/Library/Python/3.9/lib/python/site-pac